In [ ]:
# Random Starting Player - Human or AI

import numpy as np
import random
import sys
from math import inf

ROWS = 6
COLS = 7
EMPTY = 0
PLAYER = 1
AI = 2
WINDOW_SIZE = 4
MAX_DEPTH = 5

def create_board():
    return np.zeros((ROWS, COLS), dtype=int)

def is_valid_move(board, col):
    return board[0][col] == EMPTY

def get_next_open_row(board, col):
    for row in range(ROWS-1, -1, -1):
        if board[row][col] == EMPTY:
            return row
    return -1

def drop_disc(board, row, col, piece):
    board[row][col] = piece

def print_board(board):
    print("\n")
    for row in range(ROWS):
        print("|", end="")
        for col in range(COLS):
            if board[row][col] == EMPTY:
                print("   |", end="")
            elif board[row][col] == PLAYER:
                print(" X |", end="")
            elif board[row][col] == AI:
                print(" O |", end="")
        print("\n")

    print("|", end="")
    for col in range(COLS):
        print(f" {col+1} |", end="")
    print("\n")

def check_win(board, piece):
    for row in range(ROWS):
        for col in range(COLS - 3):
            if (board[row][col] == piece and
                board[row][col+1] == piece and
                board[row][col+2] == piece and
                board[row][col+3] == piece):
                return True

    for row in range(ROWS - 3):
        for col in range(COLS):
            if (board[row][col] == piece and
                board[row+1][col] == piece and
                board[row+2][col] == piece and
                board[row+3][col] == piece):
                return True

    for row in range(ROWS - 3):
        for col in range(COLS - 3):
            if (board[row][col] == piece and
                board[row+1][col+1] == piece and
                board[row+2][col+2] == piece and
                board[row+3][col+3] == piece):
                return True

    for row in range(3, ROWS):
        for col in range(COLS - 3):
            if (board[row][col] == piece and
                board[row-1][col+1] == piece and
                board[row-2][col+2] == piece and
                board[row-3][col+3] == piece):
                return True

    return False

def is_terminal_node(board):
    return check_win(board, PLAYER) or check_win(board, AI) or len(get_valid_moves(board)) == 0

def get_valid_moves(board):
    valid_moves = []
    for col in range(COLS):
        if is_valid_move(board, col):
            valid_moves.append(col)
    return valid_moves

def evaluate_window(window, piece):
    opponent = PLAYER if piece == AI else AI

    if window.count(piece) == 4:
        return 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        return 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        return 2

    if window.count(opponent) == 3 and window.count(EMPTY) == 1:
        return -4

    return 0

def score_position(board, piece):
    score = 0

    center_array = [int(board[row][COLS//2]) for row in range(ROWS)]
    center_count = center_array.count(piece)
    score += center_count * 3

    for row in range(ROWS):
        row_array = [int(board[row][col]) for col in range(COLS)]
        for col in range(COLS - 3):
            window = row_array[col:col+WINDOW_SIZE]
            score += evaluate_window(window, piece)

    for col in range(COLS):
        col_array = [int(board[row][col]) for row in range(ROWS)]
        for row in range(ROWS - 3):
            window = col_array[row:row+WINDOW_SIZE]
            score += evaluate_window(window, piece)

    for row in range(ROWS - 3):
        for col in range(COLS - 3):
            window = [board[row+i][col+i] for i in range(WINDOW_SIZE)]
            score += evaluate_window(window, piece)

    for row in range(ROWS - 3):
        for col in range(COLS - 3):
            window = [board[row+3-i][col+i] for i in range(WINDOW_SIZE)]
            score += evaluate_window(window, piece)

    return score

def minimax(board, depth, alpha, beta, maximizing_player):
    valid_moves = get_valid_moves(board)

    if depth == 0 or is_terminal_node(board):
        if is_terminal_node(board):
            if check_win(board, AI):
                return (None, 1000000)
            elif check_win(board, PLAYER):
                return (None, -1000000)
            else:
                return (None, 0)
        else:
            return (None, score_position(board, AI))

    if maximizing_player:
        value = -inf
        column = random.choice(valid_moves) if valid_moves else None
        for col in valid_moves:
            row = get_next_open_row(board, col)
            temp_board = board.copy()
            drop_disc(temp_board, row, col, AI)
            new_score = minimax(temp_board, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value

    else:
        value = inf
        column = random.choice(valid_moves) if valid_moves else None
        for col in valid_moves:
            row = get_next_open_row(board, col)
            temp_board = board.copy()
            drop_disc(temp_board, row, col, PLAYER)
            new_score = minimax(temp_board, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

board = create_board()
game_over = False

print("Connect Four Game - You are X, AI is O")
print("Enter a column number (1-7) to drop your disc")
print_board(board)

turn = random.choice([PLAYER, AI])

while not game_over:
    if turn == PLAYER:
        try:
            col = int(input("Your turn (1-7): ")) - 1

            if col < 0 or col >= COLS:
                print("Please enter a number between 1 and 7.")
                continue

            if not is_valid_move(board, col):
                print("That column is full. Try another one.")
                continue

            row = get_next_open_row(board, col)
            drop_disc(board, row, col, PLAYER)

            print_board(board)

            if check_win(board, PLAYER):
                print("You win! Congratulations!")
                game_over = True

            turn = AI

        except ValueError:
            print("Please enter a valid number.")
            continue

    else:
        col, minimax_score = minimax(board, MAX_DEPTH, -inf, inf, True)

        if col is not None and is_valid_move(board, col):
            row = get_next_open_row(board, col)
            drop_disc(board, row, col, AI)

            print(f"AI drops disc in column {col+1}")
            print_board(board)

            if check_win(board, AI):
                print("AI wins! Programmer wins, but you lose.")
                game_over = True

        turn = PLAYER

    if len(get_valid_moves(board)) == 0 and not game_over:
        print("It's a draw!")
        game_over = True

print("Game over!")